In [83]:
import heapq
import os

class BinaryTreeNode:
    def __init__(self,value,freq):
        self.value=value
        self.freq=freq
        self.left=None
        self.right=None

    def __lt__(self,other):
        return self.freq<other.freq
    
    def __eq__(self,other):
        return self.freq==other.freq

class HuffmanCoding:
    
    def __init__(self,path):
        self.path=path
        self.__heap=[]
        self.__codes={}
        self.__reverseCodes={}
    
    def __make_freq_dict(self,text):
        freq_dict=dict()
        for char in text:
            freq_dict[char]=freq_dict.get(char,0)+1
        
        return freq_dict
    
    def __buildHeap(self,freq_dict):
        
        for key in freq_dict:
            frequency=freq_dict[key]
            binary_tree_node=BinaryTreeNode(key,frequency)
            heapq.heappush(self.__heap,binary_tree_node)
            
        return
        
    
    def __buildTree(self):
        
        while len(self.__heap)>1:
            binary_tree_node1=heapq.heappop(self.__heap)
            binary_tree_node2=heapq.heappop(self.__heap)
            frequency_sum=binary_tree_node1.freq+binary_tree_node2.freq
            newNode=BinaryTreeNode(None,frequency_sum)
            newNode.left=binary_tree_node1
            newNode.right=binary_tree_node2
            heapq.heappush(self.__heap,newNode)
        
        return
  
    
    def __buildCodesHelper(self,root,curr_bits):
        
        if root is None:
            return
        
        if root.value is not None:
            self.__codes[root.value]=curr_bits
            self.__reverseCodes[curr_bits]=root.value
            return
        
        self.__buildCodesHelper(root.left,curr_bits+"0")
        self.__buildCodesHelper(root.right,curr_bits+"1")
    
    
    def __buildCodes(self):
        root=heapq.heappop(self.__heap)
        self.__buildCodesHelper(root,"")
    
    
    def __getEncodedText(self,text):
        encoded_text=""
        for char in text:
            encoded_text=encoded_text+self.__codes[char]
        
        return encoded_text
    
    
    def __getPaddedEncodedText(self,encoded_text):
        
        padded_amount= 8 - (len(encoded_text)%8)
        
        for i in range(padded_amount):
            encoded_text=encoded_text + "0"
        
        padded_bits="{0:08b}".format(padded_amount)
        padded_encoded_text=padded_bits+encoded_text
        
        return padded_encoded_text
    
    
    def __getBytesArray(self,padded_encoded_text):
        
        array=[]
        
        for i in range(0,len(padded_encoded_text),8):
            byte=padded_encoded_text[i:i+8]
            array.append(int(byte,2))
        
        return array
        
    def compress(self):
        
        #Input text file
        file_name,file_extension= os.path.splitext(self.path)
        output_path= file_name + ".bin"
        
        with open(self.path,'r+') as file, open(output_path,'wb') as output:
            
            text=file.read()
            text=text.rstrip()
            
            # 1. Make Frequency Dictionary
            freq_dict=self.__make_freq_dict(text)
            # 2. Build Heap from freq_dict
            self.__buildHeap(freq_dict)
            # 3. Build Binary Tree from Heap
            self.__buildTree()
            # 4. Build Codes from Tree
            self.__buildCodes()
            # 5. Encode Text File
            encoded_text=self.__getEncodedText(text)
            # 6. Pad encoded text file
            padded_encoded_text=self.__getPaddedEncodedText(encoded_text)
            # 7. Convert to bytes into an array
            byte_array=self.__getBytesArray(padded_encoded_text)
            final_bytes=bytes(byte_array)
            # 8. Write output bytes to bin file
            output.write(final_bytes)
        
        print("Compressed")
        return output_path
    
    def __removePadding(self,text):
        
        padding_bits=text[:8]
        text=text[8:]
        extra_bits=int(padding_bits,2)
        text_after_removing_padding=text[:-1*extra_bits]
        
        return text_after_removing_padding
    
    def __decodeText(self,text):
        
        decoded_text=""
        current_bits=""
        
        for bit in text:
            current_bits+=bit
            if current_bits in self.__reverseCodes:
                character=self.__reverseCodes[current_bits]
                decoded_text+=character
                current_bits=""
            
        return decoded_text
    
    def decompress(self,input_path):
        
        file_name,file_extension= os.path.splitext(self.path)
        output_path= file_name +"_decompressed"+".txt"
        
        with open(input_path,"rb") as file, open(output_path,"w") as output:
            
            byte=file.read(1)
            bit_string=""
            while byte:
                byte=ord(byte)
                bits=bin(byte)[2:].rjust(8,'0')
                bit_string += bits
                byte=file.read(1)
            
            actual_text= self.__removePadding(bit_string)
            decoded_text= self.__decodeText(actual_text)
            output.write(decoded_text)
            print("Decompressed")
        return
        

    

path="/Users/VizzyKaz/Documents/CodingNinjas/Python Data Structures and Algorithms/Huffman Coding/sample.txt"

h=HuffmanCoding(path)
output_path=h.compress()
h.decompress(output_path)

Compressed
Decompressed


In [81]:
a=[1,212]
c=bytes(a)
print(c)
d=ord('\xd4')
bin(d)

b'\x01\xd4'


'0b11010100'